# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [39]:
from pymongo import MongoClient
import pandas as pd
import time

# client = MongoClient("")

In [40]:
client = MongoClient("localhost:27017")

db = client["ironhack"]

db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [41]:
db.list_collection_names()

['Crunchbase', 'restaurants']

In [42]:
c = db.get_collection("Crunchbase")

In [43]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [44]:
list(c.find({"name":"Babelgum"}, {"name":1, "_id": 0}).limit(1))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [45]:
empl_5000 = {"number_of_employees": {"$gt":5000}}

projection0 = {"name":1, "number_of_employees":1, "_id":0}

list(c.find(empl_5000, projection0).sort("number_of_employees", -1).limit(20))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [46]:
after_2000 = {"founded_year": {"$gte": 2000}}
before_2005 = {"founded_year": {"$lte": 2005}}
between_2000_2005 = {"$and": [after_2000, before_2005]}
projection1 = {"_id":0, "name":1, "founded_year":1}

list(c.find(between_2000_2005, projection1).sort("founded_year", 1).limit(5))

[{'name': 'MeeVee', 'founded_year': 2000},
 {'name': 'Steorn', 'founded_year': 2000},
 {'name': 'Boonex', 'founded_year': 2000},
 {'name': 'PhotoBox', 'founded_year': 2000},
 {'name': 'AllofMP3', 'founded_year': 2000}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [56]:
valuation_100M = {"ipo.valuation_amount":{"$gt":100000000}}
before_2010 = {"founded_year": {"$lt": 2010}}
valuat_before = {"$and":[valuation_100M,before_2010]}
projection2 = {"_id":0, "name":1, "ipo":{"valuation_amount":1}}

list(c.find(valuat_before, projection2).sort("founded_year", 1).limit(5))

[{'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'QlikTech', 'ipo': {'valuation_amount': 1000000000}},
 {'name': 'Geeknet', 'ipo': {'valuation_amount': 134000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [57]:
empl_1000 = {"number_of_employees": {"$lt":1000}}
empl_before = {"$and":[empl_1000,before_2005]}
projection3 = {"_id":0, "name":1, "number_of_employees": 1}

list(c.find(empl_before, projection3).sort("number_of_employees", -1).limit(10))

[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Workday', 'number_of_employees': 900},
 {'name': 'Cornerstone OnDemand', 'number_of_employees': 881},
 {'name': 'Mozilla', 'number_of_employees': 800}]

### 6. All the companies that don't include the `partners` field.

In [60]:
list(c.find({"partners": {"$exists": False}}))

[]

### 7. All the companies that have a null type of value on the `` field.

In [61]:
list(c.find({"category_code": {"$in": ["null"]}}))

[]

In [62]:
list(c.find({"category_code": {"$exists": False}}))

[]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [64]:
empl_100 = {"number_of_employees": {"$gte":1000}}
empl_between = {"$and": [empl_100, empl_1000]}

list(c.find(empl_between, projection3).sort("number_of_employees", 1).limit(5))

[]

### 9. Order all the companies by their IPO price in a descending order.

In [73]:
list(c.find({},projection2).sort("ipo.valuation_amount", -1).limit(10))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [74]:
filter_none = {}

list(c.find(filter_none,projection3).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [75]:
second_semester = {"founded_month":{"$gt":6}}
projection4 = {"_id":0, "name":1, "founded_month":1}

list(c.find(second_semester, projection4).sort("founded_month", 1).limit(10)) # you can change the limit by adding two extra zeros :o


[{'name': 'Pando Networks', 'founded_month': 7},
 {'name': 'MeeVee', 'founded_month': 7},
 {'name': 'GigaOM', 'founded_month': 7},
 {'name': 'iContact', 'founded_month': 7},
 {'name': 'Jaiku', 'founded_month': 7},
 {'name': 'Terabitz', 'founded_month': 7},
 {'name': 'Cyworld', 'founded_month': 7},
 {'name': 'Ujogo', 'founded_month': 7},
 {'name': 'Yapta', 'founded_month': 7},
 {'name': 'Yelp', 'founded_month': 7}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [ ]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```